<a href="https://colab.research.google.com/github/nschantz21/semantic_search/blob/develop/notebooks/QueryAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [8]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

def semantic_search(query_word: str):
    """
    Function to get the most similar trend terms
    """
    cbow_wv = KeyedVectors.load("/content/drive/MyDrive/NWO Project/model1_vectors.kv", mmap='r')
    # skip gram
    skipgram_wv = KeyedVectors.load("/content/drive/MyDrive/NWO Project/model2_vectors.kv", mmap='r')
    topn=1000
    cbow_sim = cbow_wv.most_similar(query_word, topn=100)
    sg_sim = skipgram_wv.most_similar(query_word, topn=100)
    cbow_sim_frame = pd.DataFrame(cbow_sim).set_index(0)
    sg_sim_frame = pd.DataFrame(sg_sim).set_index(0)
    average_score = (cbow_sim_frame.add(sg_sim_frame, fill_value=0.0).sort_values(by=1) / 2).head(topn)
    return average_score


In [ ]:
!pip install flask-restful
!pip install flask-ngrok

In [3]:
from flask import Flask
from flask_restful import Resource, Api, reqparse
from flask_ngrok import run_with_ngrok
import ast

In [4]:
app = Flask(__name__)
run_with_ngrok(app)
api = Api(app)

In [5]:
# define our API end point for Semantic Search
class Query(Resource):
    # define the HTTP methods here
    def get(self, query_word):
        # parse arguments
        #parser = reqparse.RequestParser()
        #parser.add_argument("query_word", type=str, help="single query term for semantic search")
        #args = parser.parse_args()
        # call the semantic search function
        #related_terms = semantic_search(query_word=args[query_word]).to_dict()
        related_terms = semantic_search(query_word=query_word).to_dict()
        return {"related_terms": related_terms}, 200

In [6]:
api.add_resource(Query, "/<string:query_word>")

In [9]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://eec3-35-185-219-50.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Dec/2021 21:32:27] "GET / HTTP/1.1" 404 -
[2021-12-04 21:32:28,511] ERROR in app: Exception on /favicon.ico [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/lib/python3.7/dist-packages/flask_restful/__init__.py", line 467, in wrapper
    resp = resource(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/flask/views.py", line 89, in view
    return self.dispatch_request(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/flask_restful/__init__.py", line 582, in dispatch_request
    resp = meth(*args, **kwargs)
  File "<ipython-input-5-92999c116733>", line 11, in get
    related_terms = semantic_search(query_word=query_word).to_dict()
  File "<ipython-i